In [ ]:
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


# Grid Search CV


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
import pickle

# Load the training dataset
train_data_path = '/content/drive/MyDrive/D2/training.csv'
train_data = pd.read_csv(train_data_path)

# Load the validation dataset
val_data_path = '/content/drive/MyDrive/D2/validation.csv'
val_data = pd.read_csv(val_data_path)

# Extract features and labels from training data
X_train = train_data['statement']  # Assuming 'statement' is the column containing text data
y_train = train_data['label'].astype(int)

# Extract features and labels from validation data
X_val = val_data['statement']  # Assuming 'statement' is the column containing text data
y_val = val_data['label'].astype(int)

# Define the pipeline with TfidfVectorizer and MultinomialNB
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(max_features=500)),  # Reduce number of features for faster computation
    ('nb', MultinomialNB())
])

# Define the parameter grid for GridSearchCV
param_grid = {
    'nb__alpha': [0.1, 0.5, 1.0],  # Smoothing parameter
    'nb__fit_prior': [True, False]  # Whether to learn class prior probabilities or not
}

# Perform GridSearchCV with cross-validation on training data
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=3, verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Get the best model and evaluate it on validation data
best_pipeline = grid_search.best_estimator_
y_pred = best_pipeline.predict(X_val)

# Print evaluation metrics
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_:.2%}")
print(f"Accuracy on validation set: {accuracy_score(y_val, y_pred):.2%}")
print("Classification Report:")
print(classification_report(y_val, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))

# Save the best pipeline using pickle
model_save_path = '/content/drive/MyDrive/D2/D2_naive/gridsearch_pipeline.pkl'
with open(model_save_path, 'wb') as f:
    pickle.dump(best_pipeline, f)

print(f"Model saved at {model_save_path}")

# Load the test dataset
test_data_path = '/content/drive/MyDrive/D2/testmod.csv'
test_data = pd.read_csv(test_data_path)

# Extract features and labels from test data
X_test = test_data['statement']  # Assuming 'statement' is the column containing text data
y_test = test_data['label'].astype(int)

# Load the saved pipeline
with open(model_save_path, 'rb') as f:
    loaded_pipeline = pickle.load(f)

# Make predictions on the test data
y_test_pred = loaded_pipeline.predict(X_test)

# Print evaluation metrics for the test set
print(f"Accuracy on test set: {accuracy_score(y_test, y_test_pred):.2%}")
print("Classification Report:")
print(classification_report(y_test, y_test_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))

Fitting 3 folds for each of 6 candidates, totalling 18 fits
Best parameters: {'nb__alpha': 0.1, 'nb__fit_prior': True}
Best cross-validation score: 63.92%
Accuracy on validation set: 63.22%
Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.43      0.52       720
           1       0.63      0.80      0.70       868

    accuracy                           0.63      1588
   macro avg       0.63      0.62      0.61      1588
weighted avg       0.63      0.63      0.62      1588

Confusion Matrix:
[[311 409]
 [175 693]]
Model saved at /content/drive/MyDrive/D2/D2_naive/gridsearch_pipeline.pkl
Accuracy on test set: 39.15%
Classification Report:
              precision    recall  f1-score   support

           0       0.43      0.24      0.31       714
           1       0.37      0.59      0.46       553

    accuracy                           0.39      1267
   macro avg       0.40      0.41      0.38      1267
weighted avg       0.

# Bayes Optimization

In [ ]:
!pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 3.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from skopt import BayesSearchCV
from skopt.space import Real, Categorical
import pickle

# Paths to data files
train_data_path = '/content/drive/MyDrive/D2/training.csv'
val_data_path = '/content/drive/MyDrive/D2/testmod.csv'
test_data_path = '/content/drive/MyDrive/D2/validation.csv'
model_save_path = '/content/drive/MyDrive/D2/bayes_pipeline1.pkl'

# Load data
train_data = pd.read_csv(train_data_path)
val_data = pd.read_csv(val_data_path)
test_data = pd.read_csv(test_data_path)

# Extract features and labels from training data
X_train = train_data['statement']
y_train = train_data['label'].astype(int)

# Extract features and labels from validation data
X_val = val_data['statement']
y_val = val_data['label'].astype(int)

# Extract features and labels from test data
X_test = test_data['statement']
y_test = test_data['label'].astype(int)

# Define the pipeline with TF-IDF vectorizer and Naive Bayes model
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(max_features=500)),
    ('nb', MultinomialNB())
])

# Define the parameter space for Bayesian Optimization
param_space = {
    'nb__alpha': Real(1e-3, 1.0, prior='log-uniform'),
    'nb__fit_prior': Categorical([True, False])
}

# Perform Bayesian Optimization with cross-validation on training data
bayes_search = BayesSearchCV(
    estimator=pipeline,
    search_spaces=param_space,
    scoring='accuracy',
    cv=3,
    n_iter=20,
    verbose=2,
    n_jobs=-1
)

# Fit the Bayesian Optimization
bayes_search.fit(X_train, y_train)

# Get the best model and evaluate it on validation data
best_pipeline = bayes_search.best_estimator_
y_pred_val = best_pipeline.predict(X_val)

# Print best parameters and evaluation metrics on validation set
print(f"Best parameters: {bayes_search.best_params_}")
print(f"Best cross-validation score: {bayes_search.best_score_:.2%}")
print(f"Accuracy on validation set: {accuracy_score(y_val, y_pred_val):.2%}")
print("Classification Report (Validation Set):")
print(classification_report(y_val, y_pred_val))
print("Confusion Matrix (Validation Set):")
print(confusion_matrix(y_val, y_pred_val))

# Save the best model using pickle
with open(model_save_path, 'wb') as f:
    pickle.dump(best_pipeline, f)
print(f"Model saved at {model_save_path}")

# Evaluate the saved model on the test set
print("\nEvaluating on Test Set:")

# Load the trained model
with open(model_save_path, 'rb') as f:
    trained_model = pickle.load(f)

# Predict using the loaded model
y_pred_test = trained_model.predict(X_test)

# Evaluate performance on test set
print(f"Accuracy on test set: {accuracy_score(y_test, y_pred_test):.2%}")
print("Classification Report (Test Set):")
print(classification_report(y_test, y_pred_test))
print("Confusion Matrix (Test Set):")
print(confusion_matrix(y_test, y_pred_test))

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fi

# Population-based Training

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.base import clone
from sklearn.pipeline import Pipeline
import pickle

# Load the training dataset
train_data_path = '/content/drive/MyDrive/D2/training.csv'
train_data = pd.read_csv(train_data_path)

# Load the validation dataset
val_data_path = '/content/drive/MyDrive/D2/validation.csv'
val_data = pd.read_csv(val_data_path)

# Extract features and labels from training data
X_train = train_data['statement']  # Assuming 'statement' is the column containing text data
y_train = train_data['label'].astype(int)

# Extract features and labels from validation data
X_val = val_data['statement']  # Assuming 'statement' is the column containing text data
y_val = val_data['label'].astype(int)

# Convert text data to TF-IDF features
vectorizer = TfidfVectorizer(max_features=500)  # Reduce number of features for faster computation
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)

# Initialize population
population_size = 10
population = []

# Generate initial population with random hyperparameters
for _ in range(population_size):
    alpha = np.random.uniform(1e-3, 1.0)
    fit_prior = np.random.choice([True, False])
    model = MultinomialNB(alpha=alpha, fit_prior=fit_prior)
    model.fit(X_train_tfidf, y_train)
    population.append((model, alpha, fit_prior))

# Define number of iterations for PBT
iterations = 10

# Perform PBT
for iteration in range(iterations):
    scores = []

    # Evaluate each model in the population
    for model, alpha, fit_prior in population:
        y_pred = model.predict(X_val_tfidf)
        score = accuracy_score(y_val, y_pred)
        scores.append((score, model, alpha, fit_prior))

    # Sort population based on score
    scores.sort(reverse=True, key=lambda x: x[0])
    top_half = scores[:population_size // 2]
    bottom_half = scores[population_size // 2:]

    # Update bottom half of the population
    for i in range(len(bottom_half)):
        _, top_model, top_alpha, top_fit_prior = top_half[i]
        _, _, _, _ = bottom_half[i]

        # Clone the top model and perturb its hyperparameters
        new_alpha = np.clip(top_alpha * np.random.uniform(0.8, 1.2), 1e-3, 1.0)
        new_fit_prior = np.random.choice([True, False]) if np.random.rand() < 0.5 else top_fit_prior

        new_model = clone(top_model)
        new_model.set_params(alpha=new_alpha, fit_prior=new_fit_prior)
        new_model.fit(X_train_tfidf, y_train)

        population[population_size // 2 + i] = (new_model, new_alpha, new_fit_prior)

# Select the best model from the final population
best_model, best_alpha, best_fit_prior = max(population, key=lambda x: accuracy_score(y_val, x[0].predict(X_val_tfidf)))

# Define a pipeline with vectorizer and best model
pipeline = Pipeline([
    ('vectorizer', vectorizer),
    ('nb', best_model)
])

# Fit the pipeline on the entire training data with best hyperparameters
pipeline.fit(X_train, y_train)

# Evaluate the best model on validation data
y_pred = pipeline.predict(X_val)

# Print best hyperparameters and evaluation metrics
print(f"Best alpha: {best_alpha}")
print(f"Best fit_prior: {best_fit_prior}")
print(f"Accuracy on validation set: {accuracy_score(y_val, y_pred):.2%}")
print("Classification Report:")
print(classification_report(y_val, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))

# Save the pipeline (including vectorizer and best model) using pickle
model_save_path = '/content/drive/MyDrive/D2/naive/pbt_pipeline.pkl'
with open(model_save_path, 'wb') as f:
    pickle.dump(pipeline, f)

print(f"Pipeline saved at {model_save_path}")

Best alpha: 0.7533955582020955
Best fit_prior: False
Accuracy on validation set: 65.55%
Classification Report:
              precision    recall  f1-score   support

           0       0.61      0.68      0.64       720
           1       0.70      0.64      0.67       868

    accuracy                           0.66      1588
   macro avg       0.66      0.66      0.65      1588
weighted avg       0.66      0.66      0.66      1588

Confusion Matrix:
[[486 234]
 [313 555]]
Pipeline saved at /content/drive/MyDrive/D2/naive/pbt_pipeline.pkl


In [ ]:
import pandas as pd
import pickle
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Load the test dataset
test_data_path = '/content/drive/MyDrive/D2/validation.csv'
test_data = pd.read_csv(test_data_path)

# Assuming 'statement' is the column containing text data in the test dataset
X_test = test_data['statement']
y_test = test_data['label'].astype(int)

# Load the saved pipeline
model_load_path = '/content/drive/MyDrive/D2/naive/pbt_pipeline.pkl'
with open(model_load_path, 'rb') as f:
    pipeline = pickle.load(f)

# Make predictions using the pipeline
y_pred = pipeline.predict(X_test)

# Evaluate the predictions
print(f"Accuracy on test set: {accuracy_score(y_test, y_pred):.2%}")
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy on test set: 65.55%
Classification Report:
              precision    recall  f1-score   support

           0       0.61      0.68      0.64       720
           1       0.70      0.64      0.67       868

    accuracy                           0.66      1588
   macro avg       0.66      0.66      0.65      1588
weighted avg       0.66      0.66      0.66      1588

Confusion Matrix:
[[486 234]
 [313 555]]


# Genetic Algorithms

In [ ]:
!pip install deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from deap import base, creator, tools, algorithms
import random
import pickle

# Load the training dataset
train_data_path = '/content/drive/MyDrive/D2/training.csv'
train_data = pd.read_csv(train_data_path)

# Load the validation dataset
val_data_path = '/content/drive/MyDrive/D2/validation.csv'
val_data = pd.read_csv(val_data_path)

# Extract features and labels from training data
X_train = train_data['statement']  # Assuming 'statement' is the column containing text data
y_train = train_data['label'].astype(int)  # Assuming 'label' is the column containing labels

# Extract features and labels from validation data
X_val = val_data['statement']  # Assuming 'statement' is the column containing text data
y_val = val_data['label'].astype(int)  # Assuming 'label' is the column containing labels

# Convert text data to TF-IDF features
vectorizer = TfidfVectorizer(max_features=500)  # Reduce number of features for faster computation
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)

# Define the evaluation function for genetic algorithm
def evaluate(individual):
    alpha = individual[0]
    fit_prior = bool(individual[1])
    model = MultinomialNB(alpha=alpha, fit_prior=fit_prior)
    model.fit(X_train_tfidf, y_train)
    y_pred = model.predict(X_val_tfidf)
    accuracy = accuracy_score(y_val, y_pred)
    return (accuracy,)

# Set up the genetic algorithm
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, 1e-3, 1.0)
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_float, toolbox.attr_bool), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutPolynomialBounded, low=[1e-3, 0], up=[1.0, 1], eta=0.1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

# Genetic Algorithm parameters
population_size = 20
generations = 40
cxpb, mutpb = 0.5, 0.2

# Initialize population
population = toolbox.population(n=population_size)

# Run the Genetic Algorithm
result_population, logbook = algorithms.eaSimple(population, toolbox, cxpb, mutpb, generations,
                                                 stats=None, halloffame=None, verbose=True)

# Select the best individual
best_individual = tools.selBest(result_population, k=1)[0]
best_alpha = best_individual[0]
best_fit_prior = bool(best_individual[1])

# Train the best model on the training data
best_model = MultinomialNB(alpha=best_alpha, fit_prior=best_fit_prior)
best_model.fit(X_train_tfidf, y_train)

# Define a pipeline with vectorizer and best model
pipeline = Pipeline([
    ('vectorizer', vectorizer),
    ('nb', best_model)
])

# Fit the pipeline on the entire training data with best hyperparameters
pipeline.fit(X_train, y_train)

# Evaluate the best model on validation data
y_pred = pipeline.predict(X_val)

# Print best hyperparameters and evaluation metrics
print(f"Best alpha: {best_alpha}")
print(f"Best fit_prior: {best_fit_prior}")
print(f"Accuracy on validation set: {accuracy_score(y_val, y_pred):.2%}")
print("Classification Report:")
print(classification_report(y_val, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))

# Save the pipeline (including vectorizer and best model) using pickle
model_save_path = '/content/drive/MyDrive/D2/naive/genetic_pipeline.pkl'
with open(model_save_path, 'wb') as f:
    pickle.dump(pipeline, f)

print(f"Pipeline saved at {model_save_path}")

gen	nevals
0  	20    
1  	12    
2  	11    
3  	12    
4  	11    
5  	17    
6  	6     
7  	9     
8  	16    
9  	10    
10 	11    
11 	14    
12 	12    
13 	11    
14 	10    
15 	14    
16 	15    
17 	12    
18 	10    
19 	14    
20 	15    
21 	11    
22 	12    
23 	11    
24 	13    
25 	12    
26 	11    
27 	11    
28 	10    
29 	7     
30 	14    
31 	8     
32 	10    
33 	14    
34 	14    
35 	9     
36 	8     
37 	16    
38 	7     
39 	6     
40 	14    
Best alpha: 0.6927793527466417
Best fit_prior: False
Accuracy on validation set: 65.62%
Classification Report:
              precision    recall  f1-score   support

           0       0.61      0.68      0.64       720
           1       0.70      0.64      0.67       868

    accuracy                           0.66      1588
   macro avg       0.66      0.66      0.66      1588
weighted avg       0.66      0.66      0.66      1588

Confusion Matrix:
[[487 233]
 [313 555]]
Pipeline saved at /content/drive/MyDrive/D2/naive/genetic_p

In [ ]:
# Load the test dataset
test_data_path = '/content/drive/MyDrive/D2/validation.csv'
test_data = pd.read_csv(test_data_path)

# Assuming 'statement' is the column containing text data in the test dataset
X_test = test_data['statement']
y_test = test_data['label'].astype(int)

# Load the saved pipeline
model_load_path = '/content/drive/MyDrive/D2/naive/genetic_pipeline.pkl'
with open(model_load_path, 'rb') as f:
    pipeline = pickle.load(f)

# Make predictions using the pipeline
y_pred = pipeline.predict(X_test)

# Evaluate the predictions
print(f"Accuracy on test set: {accuracy_score(y_test, y_pred):.2%}")
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Accuracy on test set: 65.62%
Classification Report:
              precision    recall  f1-score   support

           0       0.61      0.68      0.64       720
           1       0.70      0.64      0.67       868

    accuracy                           0.66      1588
   macro avg       0.66      0.66      0.66      1588
weighted avg       0.66      0.66      0.66      1588

Confusion Matrix:
[[487 233]
 [313 555]]


# Hyperband

In [ ]:
!pip install hyperopt

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from hyperopt import hp, tpe, Trials, fmin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import pickle

# Load the training dataset
train_data_path = '/content/drive/MyDrive/D2/training.csv'
train_data = pd.read_csv(train_data_path)

# Load the validation dataset
val_data_path = '/content/drive/MyDrive/D2/validation.csv'
val_data = pd.read_csv(val_data_path)

# Extract features and labels from training data
X_train = train_data['statement']  # Assuming 'statement' is the column containing text data
y_train = train_data['label'].astype(int)  # Assuming 'label' is the column containing labels

# Extract features and labels from validation data
X_val = val_data['statement']  # Assuming 'statement' is the column containing text data
y_val = val_data['label'].astype(int)  # Assuming 'label' is the column containing labels

# Convert text data to TF-IDF features
vectorizer = TfidfVectorizer(max_features=500)  # Reduce number of features for faster computation
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)

# Define the evaluation function for hyperopt
def evaluate(params):
    alpha = params['alpha']
    fit_prior = params['fit_prior']
    model = MultinomialNB(alpha=alpha, fit_prior=fit_prior)
    model.fit(X_train_tfidf, y_train)
    y_pred = model.predict(X_val_tfidf)
    accuracy = accuracy_score(y_val, y_pred)
    return -accuracy  # Minimize negative accuracy (maximize accuracy)

# Define the search space
space = {
    'alpha': hp.loguniform('alpha', np.log(1e-3), np.log(1.0)),  # Smoothing parameter
    'fit_prior': hp.choice('fit_prior', [True, False])  # Whether to learn class prior probabilities or not
}

# Perform hyperparameter optimization with Hyperopt
trials = Trials()
best = fmin(fn=evaluate,
            space=space,
            algo=tpe.suggest,
            max_evals=100,  # Number of trials
            trials=trials)

# Get the best hyperparameters
best_alpha = best['alpha']
best_fit_prior = [True, False][best['fit_prior']]

# Train the best model on the full training data with the best hyperparameters
best_model = MultinomialNB(alpha=best_alpha, fit_prior=best_fit_prior)
best_model.fit(X_train_tfidf, y_train)

# Define a pipeline with vectorizer and best model
pipeline = Pipeline([
    ('vectorizer', vectorizer),
    ('nb', best_model)
])

# Fit the pipeline on the entire training data with best hyperparameters
pipeline.fit(X_train, y_train)

# Evaluate the best model on validation data
y_pred = pipeline.predict(X_val)

# Print best hyperparameters and evaluation metrics
print(f"Best alpha: {best_alpha}")
print(f"Best fit_prior: {best_fit_prior}")
print(f"Accuracy on validation set: {accuracy_score(y_val, y_pred):.2%}")
print("Classification Report:")
print(classification_report(y_val, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))

# Save the pipeline (including vectorizer and best model) using pickle
model_save_path = '/content/drive/MyDrive/D2/naive/hyperband_pipeline.pkl'
with open(model_save_path, 'wb') as f:
    pickle.dump(pipeline, f)

print(f"Pipeline saved at {model_save_path}")

100%|██████████| 100/100 [00:06<00:00, 15.98trial/s, best loss: -0.6561712846347607]
Best alpha: 0.7247052253441295
Best fit_prior: False
Accuracy on validation set: 65.62%
Classification Report:
              precision    recall  f1-score   support

           0       0.61      0.68      0.64       720
           1       0.70      0.64      0.67       868

    accuracy                           0.66      1588
   macro avg       0.66      0.66      0.66      1588
weighted avg       0.66      0.66      0.66      1588

Confusion Matrix:
[[487 233]
 [313 555]]
Pipeline saved at /content/drive/MyDrive/D2/naive/hyperband_pipeline.pkl


In [ ]:
# Load the test dataset
test_data_path = '/content/drive/MyDrive/D2/testmod.csv'
test_data = pd.read_csv(test_data_path)

# Assuming 'statement' is the column containing text data in the test dataset
X_test = test_data['statement']
y_test = test_data['label'].astype(int)

# Load the saved pipeline
model_load_path = '/content/drive/MyDrive/D2/naive/hyperband_pipeline.pkl'
with open(model_load_path, 'rb') as f:
    pipeline = pickle.load(f)

# Make predictions using the pipeline
y_pred = pipeline.predict(X_test)

# Evaluate the predictions
print(f"Accuracy on test set: {accuracy_score(y_test, y_pred):.2%}")
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Accuracy on test set: 38.75%
Classification Report:
              precision    recall  f1-score   support

           0       0.45      0.40      0.42       714
           1       0.32      0.37      0.35       553

    accuracy                           0.39      1267
   macro avg       0.39      0.39      0.39      1267
weighted avg       0.40      0.39      0.39      1267

Confusion Matrix:
[[284 430]
 [346 207]]


# BERT + Grid

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from transformers import BertTokenizer, BertModel
import pickle

In [ ]:
# Load the test dataset
test_data_path = '/content/drive/MyDrive/D2/testmod.csv'
test_data = pd.read_csv(test_data_path)

# Extract features and labels from test data
X_test = test_data['statement']
y_test = test_data['label'].astype(int)

In [ ]:
import torch

# Load the state dictionary
state_dict_path = '/content/drive/MyDrive/D2/bert_model/bert_model.pth'
state_dict = torch.load(state_dict_path)

# Print the keys in the state dictionary
print("State Dictionary Keys:")
for key in state_dict.keys():
    print(key)

State Dictionary Keys:
bert.embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.weight
bert.embeddings.LayerNorm.bias
bert.encoder.layer.0.attention.self.query.weight
bert.encoder.layer.0.attention.self.query.bias
bert.encoder.layer.0.attention.self.key.weight
bert.encoder.layer.0.attention.self.key.bias
bert.encoder.layer.0.attention.self.value.weight
bert.encoder.layer.0.attention.self.value.bias
bert.encoder.layer.0.attention.output.dense.weight
bert.encoder.layer.0.attention.output.dense.bias
bert.encoder.layer.0.attention.output.LayerNorm.weight
bert.encoder.layer.0.attention.output.LayerNorm.bias
bert.encoder.layer.0.intermediate.dense.weight
bert.encoder.layer.0.intermediate.dense.bias
bert.encoder.layer.0.output.dense.weight
bert.encoder.layer.0.output.dense.bias
bert.encoder.layer.0.output.LayerNorm.weight
bert.encoder.layer.0.output.LayerNorm.bias
bert.encoder.layer.1.attention.sel

In [ ]:
import torch
from transformers import BertTokenizer, BertModel

# Define the BERTClassifier class according to the saved model architecture
class BERTClassifier(torch.nn.Module):
    def __init__(self, bert_model):
        super(BERTClassifier, self).__init__()
        self.bert = bert_model
        self.classifier = torch.nn.Linear(768, 2)  # This should match the saved model's classifier

    def forward(self, input_ids, attention_mask=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.classifier(pooled_output)
        return torch.nn.functional.softmax(x, dim=1)

# Load the BERT model and tokenizer
bert_model = BertModel.from_pretrained('bert-base-uncased')
classifier_model = BERTClassifier(bert_model)

# Load the state dict
state_dict_path = '/content/drive/MyDrive/D2/bert_model/bert_model.pth'
state_dict = torch.load(state_dict_path)

# Load the state dict into the model
classifier_model.load_state_dict(state_dict)
classifier_model.eval()

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def predict_bert(texts, batch_size=32):
    predictions = []
    num_batches = int(np.ceil(len(texts) / batch_size))
    for i in range(num_batches):
        batch_texts = texts[i*batch_size:(i+1)*batch_size]
        batch_texts = batch_texts.tolist()
        inputs = tokenizer(batch_texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = bert_model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
        logits = outputs
        batch_predictions = torch.argmax(logits, dim=1).numpy()
        predictions.extend(batch_predictions)
    return np.array(predictions)

In [ ]:
# Load the saved Naive Bayes pipeline
model_save_path = '/content/drive/MyDrive/D2/D2_naive/gridsearch_pipeline.pkl'
with open(model_save_path, 'rb') as f:
    nb_pipeline = pickle.load(f)

In [ ]:
def predict_bert(texts, batch_size=32):
    predictions = []
    num_batches = int(np.ceil(len(texts) / batch_size))
    for i in range(num_batches):
        batch_texts = texts[i*batch_size:(i+1)*batch_size]
        batch_texts = batch_texts.tolist()
        inputs = tokenizer(batch_texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = classifier_model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask']) # Use classifier_model instead of bert_model
        logits = outputs # logits are now the output of the classifier
        batch_predictions = torch.argmax(logits, dim=1).numpy()
        predictions.extend(batch_predictions)
    return np.array(predictions)

In [ ]:
# Get predictions from both models
nb_predictions = nb_pipeline.predict(X_test)
bert_predictions = predict_bert(X_test)

# Combine predictions using majority voting
combined_predictions = np.array([np.bincount([nb_pred, bert_pred]).argmax() for nb_pred, bert_pred in zip(nb_predictions, bert_predictions)])

# Print evaluation metrics for the test set
print(f"Accuracy on test set: {accuracy_score(y_test, combined_predictions):.2%}")
print("Classification Report:")
print(classification_report(y_test, combined_predictions))
print("Confusion Matrix:")
print(confusion_matrix(y_test, combined_predictions))

Accuracy on test set: 39.38%
Classification Report:
              precision    recall  f1-score   support

           0       0.44      0.26      0.33       714
           1       0.37      0.57      0.45       553

    accuracy                           0.39      1267
   macro avg       0.40      0.41      0.39      1267
weighted avg       0.41      0.39      0.38      1267

Confusion Matrix:
[[186 528]
 [240 313]]


# Test with Batch Size

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from transformers import BertTokenizer, BertModel
import pickle

# Load the Naive Bayes pipeline
model_save_path = '/content/drive/MyDrive/D2/D2_naive/gridsearch_pipeline.pkl'
with open(model_save_path, 'rb') as f:
    nb_pipeline = pickle.load(f)

# Load the test dataset
test_data_path = '/content/drive/MyDrive/D2/testmod.csv'
test_data = pd.read_csv(test_data_path)

# Sample a subset from the test data
subset_size = 1000  # Define the size of the subset
subset_data = test_data.sample(n=subset_size, random_state=42)

# Extract features and labels from the subset
X_test = subset_data['statement']
y_test = subset_data['label'].astype(int)

# Load and prepare BERT model and tokenizer
class BERTClassifier(torch.nn.Module):
    def __init__(self, bert_model):
        super(BERTClassifier, self).__init__()
        self.bert = bert_model
        self.classifier = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.classifier(pooled_output)
        return x

# Load BERT model
bert_model = BertModel.from_pretrained('bert-base-uncased')
bert_model = BERTClassifier(bert_model)
bert_model.load_state_dict(torch.load('/content/drive/MyDrive/D2/bert_model/bert_model.pth'))
bert_model.eval()

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define prediction functions
def predict_bert(texts):
    texts = texts.tolist()  # Convert to a list of strings
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = bert_model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
    logits = outputs
    return torch.argmax(logits, dim=1).numpy()

def predict_naive_bayes(texts):
    return nb_pipeline.predict(texts)

# Batch processing
batch_size = 100  # Adjust batch size according to your memory limits
num_batches = int(np.ceil(len(X_test) / batch_size))

combined_predictions = []

for i in range(num_batches):
    start = i * batch_size
    end = min((i + 1) * batch_size, len(X_test))
    batch_texts = X_test[start:end]

    bert_batch_predictions = predict_bert(batch_texts)
    nb_batch_predictions = predict_naive_bayes(batch_texts)

    for nb_pred, bert_pred in zip(nb_batch_predictions, bert_batch_predictions):
        combined_predictions.append(np.bincount([nb_pred, bert_pred]).argmax())

    # Clear cache to free up memory
    torch.cuda.empty_cache()

combined_predictions = np.array(combined_predictions)

# Calculate and print accuracy
accuracy = accuracy_score(y_test, combined_predictions)
print(f"Combined Accuracy: {accuracy:.2%}")

# Print additional metrics
print("Classification Report:")
print(classification_report(y_test, combined_predictions))
print("Confusion Matrix:")
print(confusion_matrix(y_test, combined_predictions))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Combined Accuracy: 38.70%
Classification Report:
              precision    recall  f1-score   support

           0       0.45      0.26      0.33       577
           1       0.36      0.56      0.44       423

    accuracy                           0.39      1000
   macro avg       0.40      0.41      0.38      1000
weighted avg       0.41      0.39      0.37      1000

Confusion Matrix:
[[150 427]
 [186 237]]
